In [1]:
import pandas as pd
import numpy as np
import glob

import os

from sklearn.metrics import cohen_kappa_score

from scipy.stats import spearmanr, kendalltau

import re

import matplotlib.pyplot as plt

In [2]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 200)
pd.options.display.float_format = '{:,.6f}'.format

In [3]:
ANNOTATIONS_FOLDER="anotações_humanas"

In [4]:
a1_df = pd.read_csv(os.path.join(ANNOTATIONS_FOLDER, "admin_240_annotations_with_questions.tsv"), sep='\t')

In [5]:
a2_df = pd.read_csv(os.path.join(ANNOTATIONS_FOLDER, "Eduardo_240_annotations_with_questions.tsv"), sep='\t')

In [6]:
a3_df = pd.read_csv(os.path.join(ANNOTATIONS_FOLDER, "Leodecio_240_annotations_with_questions.tsv"), sep='\t')

In [7]:
score_mapper = {
    0:0,
    1:0,
    2:1,
    3:2
}

In [8]:
score_mapper_original = {
    0:0,
    1:1,
    2:1,
    3:2
}

In [9]:
score_mapper_for_2 = {
    0:0,
    1:0,
    2:1,
    3:1    
}

In [10]:
a1_df['03_score'] = a1_df['score'].map(score_mapper)
a2_df['03_score'] = a2_df['score'].map(score_mapper)
a3_df['03_score'] = a3_df['score'].map(score_mapper)

In [11]:
a1_df['02_score'] = a1_df['score'].map(score_mapper_for_2)
a2_df['02_score'] = a2_df['score'].map(score_mapper_for_2)
a3_df['02_score'] = a3_df['score'].map(score_mapper_for_2)

In [ ]:
a1_df.iloc[126]

In [ ]:
a2_df[a2_df['query'] == "Quando é o dia Mundial da Alimentação?"]

In [12]:
def check_agreement_per_questions(evaluation_a, evaluation_b, prefix=None):
    merged_df = evaluation_a.merge(evaluation_b, left_on='doccano_id', right_on='doccano_id')[['query_x', 'passage_x', 'passage_id_x', 'score_x', 'score_y']]
    
    correlations = []
    
    for group_name, group_df in merged_df.groupby('query_x', sort=False):
        correlations.append({'query': group_name,
                             'cohen_kappa{}'.format(prefix): cohen_kappa_score(group_df['score_x'], group_df['score_y'])})
        
    return merged_df, pd.DataFrame(correlations).fillna(1.0)

In [13]:
def plot_correlation_data_consolidated(correlation_df, columns_to_plot, plot_title):
    
    fig = plt.figure(figsize=[10, 10])

    fig.suptitle(plot_title, y=0.91)

    plt.grid()

    all_boxplots = plt.boxplot(correlation_df[columns_to_plot], patch_artist=True, vert=False, meanline=True, showmeans=True)

    colors = ['pink', 'lightblue', 'lightgreen']
    
    all_boxes = all_boxplots['boxes']
    
    print(all_boxes)
    
    for i, which_box in enumerate(all_boxes):
        which_box.set_facecolor(colors[i // 3])
    
    
    plt.yticks(range(1, len(columns_to_plot) + 1), columns_to_plot)
    plt.xticks(np.arange(-0.3, 1.0, 0.1))
    
    plt.legend(handles=all_boxes[::3], labels=["Humanos x Humanos", "Humanos x GPT3.5", "Humanos x GPT4"], bbox_to_anchor=(1.0, 1.0))

    plt.show()

In [14]:
def plot_correlation_data(correlation_df, columns_to_plot, plot_title):
    
    fig = plt.figure(figsize=[15, 40])

    fig.suptitle(plot_title, y=0.91)

    plt.grid()

    all_boxplots = plt.boxplot(correlation_df[columns_to_plot].to_numpy().transpose(), patch_artist=True, vert=False, meanline=True, showmeans=True)

    colors = ['pink', 'lightblue', 'lightgreen', 'lightyellow'][::-1]
    
    all_boxes = all_boxplots['boxes']
    
#     print(all_boxes)
    
    for i, which_box in enumerate(all_boxes):
        which_box.set_facecolor(colors[i % 4])

        
    
    plt.yticks(range(4, correlation_df.shape[0] + 1, 4), correlation_df.iloc[::4]['query'])
    plt.xticks(np.arange(-0.3, 1.0, 0.1))

    plt.show()

## Read the passage evaluations

In [15]:
gpt4_df = pd.read_csv(os.path.join(ANNOTATIONS_FOLDER, "gpt_4_evaluations_240_samples_2nd_round_with_questions.tsv"), sep='\t')

In [16]:
gpt4_df['03_score'] = gpt4_df['score'].map(score_mapper)

In [17]:
gpt4_df['03_score_2'] = gpt4_df['score'].map(score_mapper_original)

In [18]:
gpt4_df['02_score'] = gpt4_df['score'].map(score_mapper_for_2)

In [35]:
gpt4_2nd_cot_2_6_df = pd.read_csv(os.path.join(ANNOTATIONS_FOLDER, "gpt_4_evaluations_120_samples_2nd_queries_set_prompt_v2.6.tsv"), sep='\t')

In [70]:
gpt4_1st_cot_2_6_df = pd.read_csv(os.path.join(ANNOTATIONS_FOLDER, "gpt_4_evaluations_120_samples_1st_queries_set_prompt_v2.6.tsv"), sep='\t')

In [19]:
gpt4_2nd_03_score_2_7_df = pd.read_csv(os.path.join(ANNOTATIONS_FOLDER, "gpt_4_evaluations_120_samples_2nd_queries_set_prompt_v2.7.tsv"), sep='\t')

In [20]:
gpt4_2nd_03_score_2_8_df = pd.read_csv(os.path.join(ANNOTATIONS_FOLDER, "gpt_4_evaluations_120_samples_2nd_queries_set_prompt_v2.8.tsv"), sep='\t')

In [21]:
gpt4_2nd_03_score_2_9_df = pd.read_csv(os.path.join(ANNOTATIONS_FOLDER, "gpt_4_evaluations_120_samples_2nd_queries_set_prompt_v2.9.tsv"), sep='\t')

In [22]:
gpt4_1st_03_score_2_7_df = pd.read_csv(os.path.join(ANNOTATIONS_FOLDER, "gpt_4_evaluations_120_samples_1st_queries_set_prompt_v2.7.tsv"), sep='\t')

In [23]:
gpt4_1st_03_score_2_9_df = pd.read_csv(os.path.join(ANNOTATIONS_FOLDER, "gpt_4_evaluations_120_samples_1st_queries_set_prompt_v2.9.tsv"), sep='\t')

In [40]:
gpt4_10_scores_df = pd.read_csv(os.path.join(ANNOTATIONS_FOLDER, "gpt_4_evaluations_240_samples_2nd_round_with_questions.tsv"), sep='\t')

In [42]:
np.unique(gpt4_10_scores_df['score'].to_numpy(), return_counts=True)

(array([0, 1, 2, 3]), array([31, 74, 59, 76]))

In [24]:
gpt4_2nd_03_score_2_7_df['02_score'] = gpt4_2nd_03_score_2_7_df['score'].map(score_mapper_for_2)
gpt4_2nd_03_score_2_8_df['02_score'] = gpt4_2nd_03_score_2_8_df['score'].map(score_mapper_for_2)
gpt4_2nd_03_score_2_9_df['02_score'] = gpt4_2nd_03_score_2_9_df['score'].map(score_mapper_for_2)

gpt4_1st_03_score_2_7_df['02_score'] = gpt4_1st_03_score_2_7_df['score'].map(score_mapper_for_2)
gpt4_1st_03_score_2_9_df['02_score'] = gpt4_1st_03_score_2_9_df['score'].map(score_mapper_for_2)

In [36]:
gpt4_2nd_cot_2_6_df['02_score'] = gpt4_2nd_cot_2_6_df['score'].map(score_mapper_for_2)

In [71]:
gpt4_1st_cot_2_6_df['02_score'] = gpt4_1st_cot_2_6_df['score'].map(score_mapper_for_2)

In [25]:
gpt4_03_score_2_7_df = pd.concat([gpt4_1st_03_score_2_7_df, gpt4_2nd_03_score_2_7_df], axis=0).reset_index(drop=True)

In [26]:
gpt4_03_score_2_7_df

,doccano_id,query,passage,passage_id,score,reasoning,usage,cost,duration,02_score
0,4101,Onde está localizada a Praça XV de Novembro?,"Praça XV (Rio de Janeiro) – Wikipédia, a enciclopédia livre [ocultar] 5 línguas Praça XV (Rio de Janeiro) Origem: Wikipédia, a enciclopédia livre. Coordenadas: 22° 54′ 10″ S, 43° 10′ 24″ W Nota: Este artigo é sobre a praça da cidade do Rio de Janeiro. Para praças de outros lugares, veja Praça XV de Novembro. Esta página cita fontes, mas estas não cobrem todo o conteúdo. Ajude a inserir referências. Conteúdo não verificável poderá ser removido .— Encontre fontes: Google (notícias, livros e acadêmico)(Maio de 2019) A praça e as edificações das imediações. Trecho do Elevado da Perimetral que passava sobre a praça, em 2011. A Praça XV, também conhecida como Praça XV de Novembro, Praça Quinze de Novembro ou Praça Quinze, é uma praça situada no bairro do Centro, na Zona Central da cidade do Rio de Janeiro. Além de estar localizada na Rua Primeiro de Março, a praça integra a Orla Conde, um passeio público que margeia a Baía de Guanabara . A praça, que existe desde o século XVI, foi reinaugurada no dia 29 de maio de 2016 após ser revitalizada.[1] A revitalização da praça foi feita durante a gestão do prefeito Eduardo Paes no âmbito do Porto Maravilha, uma operação urbana que visa revitalizar a Zona Portuária do Rio de Janeiro.",clueweb22-pt0001-14-16263_0,2,"A passagem responde claramente e diretamente à pergunta, indicando que a Praça XV de Novembro está localizada no bairro do Centro, na Zona Central da cidade do Rio de Janeiro","{'prompt_tokens': 1067, 'completion_tokens': 55, 'total_tokens': 1122}",0.035310,4.101760,1
1,4102,Onde está localizada a Praça XV de Novembro?,"Rua XV de Novembro, 818 Centro, Pomerode, Santa Catarina 89107-000 Brasil Tela cheia O melhor por perto Restaurantes 100 dentro de 5 km Tarthurel Pizzaria 317 56 m $$ - $$$ • Italiana • Pizza Restaurante Torten Paradies 1.409 0.5 km $$ - $$$ • Docerias • Alemã • Brasileira Mon Cheri Bistro @ Casa Romena 106 0.7 km $$ - $$$ • Mediterrânea • Saudável • Leste europeia Veja tudo Atrações 24 dentro de 10 km Praça Jorge Lacerda 58 0.1 km Pontos de interesse Parque Municipal de Eventos 25 0.3 km Centros de convenção Parque Educativo Vila Encantada 1.057 0.9 km Parques Veja tudo Contribua Faça uma avaliação Carregue uma foto Avaliações Mais recente 4.5 722 avaliações Excelente 379 Muito boa 266 Razoável 68 Ruim 9 Horrível 1 Thais Leme, SP 161 contribuições Parada obrigatória para fotos mar de 2022 Quando eu via as fotos achava que era bem maior. O portal é muito bonito e tem um centro e informações turísticas. Estava no período da Osterfest, então estava todo com a decoração de Páscoa. Feita em 29 de abril de 2022 Esta avaliação representa a opinião subjetiva de um membro do Tripadvisor, e não da TripAdvisor LLC. daninoguei São Paulo, SP 2.143 contribuições MUITAS PESSOAS PARA TIRAR FOTOS mar de 2022 Portal turístico da cidade, local de parada para fotos com a representação da arquitetura local.",clueweb22-pt0000-27-16948_2,2,"A passagem responde à pergunta de forma clara e direta, indicando que a Praça XV de Novembro está localizada em Pomerode, Santa Catarina, Brasil","{'prompt_tokens': 1083, 'completion_tokens': 51, 'total_tokens': 1134}",0.035550,3.668287,1
2,4103,Onde está localizada a Praça XV de Novembro?,"A praça recebeu esse nome por homenagear o dia 15 de novembro de 1889, marcado pela Proclamação da República do Brasil. Nesse dia, foi instaurada a república federativa presidencialista no Brasil, em substituição à monarquia constitucional parlamentarista em vigor desde 1822. Índice 1 História 1.1 Período colonial 1.2 Século XIX 1.3 Século XX 1.4 Século XXI 2 Esculturas e Monumentos 2.1 Chafariz do Mestre Valentim 2.2 Estátua Equestre de Dom João VI 2.3 Estátua Equestre do General Osório 2.4 Estátua do Almirante Negro 2.5 Piskate 3 Transportes 3.1 Modais de transporte público 3.2 Elevado da Perimetral 3.3 Mergulhão da Praça XV 4 Po

In [27]:
gpt4_03_score_2_7_df.to_csv(os.path.join(ANNOTATIONS_FOLDER, "gpt_4_evaluations_240_samples_prompt_v2.7.tsv"), sep='\t', index=False)

In [28]:
gpt4_03_score_2_9_df = pd.concat([gpt4_1st_03_score_2_9_df, gpt4_2nd_03_score_2_9_df], axis=0).reset_index(drop=True)

gpt4_03_score_2_9_df

,doccano_id,query,passage,passage_id,score,reasoning,usage,cost,duration,02_score
0,4101,Onde está localizada a Praça XV de Novembro?,"Praça XV (Rio de Janeiro) – Wikipédia, a enciclopédia livre [ocultar] 5 línguas Praça XV (Rio de Janeiro) Origem: Wikipédia, a enciclopédia livre. Coordenadas: 22° 54′ 10″ S, 43° 10′ 24″ W Nota: Este artigo é sobre a praça da cidade do Rio de Janeiro. Para praças de outros lugares, veja Praça XV de Novembro. Esta página cita fontes, mas estas não cobrem todo o conteúdo. Ajude a inserir referências. Conteúdo não verificável poderá ser removido .— Encontre fontes: Google (notícias, livros e acadêmico)(Maio de 2019) A praça e as edificações das imediações. Trecho do Elevado da Perimetral que passava sobre a praça, em 2011. A Praça XV, também conhecida como Praça XV de Novembro, Praça Quinze de Novembro ou Praça Quinze, é uma praça situada no bairro do Centro, na Zona Central da cidade do Rio de Janeiro. Além de estar localizada na Rua Primeiro de Março, a praça integra a Orla Conde, um passeio público que margeia a Baía de Guanabara . A praça, que existe desde o século XVI, foi reinaugurada no dia 29 de maio de 2016 após ser revitalizada.[1] A revitalização da praça foi feita durante a gestão do prefeito Eduardo Paes no âmbito do Porto Maravilha, uma operação urbana que visa revitalizar a Zona Portuária do Rio de Janeiro.",clueweb22-pt0001-14-16263_0,2,"A passagem responde claramente e de forma completa à pergunta, indicando que a Praça XV de Novembro está localizada no bairro do Centro, na Zona Central da cidade do Rio de Janeiro","{'prompt_tokens': 1040, 'completion_tokens': 56, 'total_tokens': 1096}",0.034560,3.822135,1
1,4102,Onde está localizada a Praça XV de Novembro?,"Rua XV de Novembro, 818 Centro, Pomerode, Santa Catarina 89107-000 Brasil Tela cheia O melhor por perto Restaurantes 100 dentro de 5 km Tarthurel Pizzaria 317 56 m $$ - $$$ • Italiana • Pizza Restaurante Torten Paradies 1.409 0.5 km $$ - $$$ • Docerias • Alemã • Brasileira Mon Cheri Bistro @ Casa Romena 106 0.7 km $$ - $$$ • Mediterrânea • Saudável • Leste europeia Veja tudo Atrações 24 dentro de 10 km Praça Jorge Lacerda 58 0.1 km Pontos de interesse Parque Municipal de Eventos 25 0.3 km Centros de convenção Parque Educativo Vila Encantada 1.057 0.9 km Parques Veja tudo Contribua Faça uma avaliação Carregue uma foto Avaliações Mais recente 4.5 722 avaliações Excelente 379 Muito boa 266 Razoável 68 Ruim 9 Horrível 1 Thais Leme, SP 161 contribuições Parada obrigatória para fotos mar de 2022 Quando eu via as fotos achava que era bem maior. O portal é muito bonito e tem um centro e informações turísticas. Estava no período da Osterfest, então estava todo com a decoração de Páscoa. Feita em 29 de abril de 2022 Esta avaliação representa a opinião subjetiva de um membro do Tripadvisor, e não da TripAdvisor LLC. daninoguei São Paulo, SP 2.143 contribuições MUITAS PESSOAS PARA TIRAR FOTOS mar de 2022 Portal turístico da cidade, local de parada para fotos com a representação da arquitetura local.",clueweb22-pt0000-27-16948_2,2,"A passagem indica que a Praça XV de Novembro está localizada em Pomerode, Santa Catarina, Brasil","{'prompt_tokens': 1056, 'completion_tokens': 37, 'total_tokens': 1093}",0.033900,3.115484,1
2,4103,Onde está localizada a Praça XV de Novembro?,"A praça recebeu esse nome por homenagear o dia 15 de novembro de 1889, marcado pela Proclamação da República do Brasil. Nesse dia, foi instaurada a república federativa presidencialista no Brasil, em substituição à monarquia constitucional parlamentarista em vigor desde 1822. Índice 1 História 1.1 Período colonial 1.2 Século XIX 1.3 Século XX 1.4 Século XXI 2 Esculturas e Monumentos 2.1 Chafariz do Mestre Valentim 2.2 Estátua Equestre de Dom João VI 2.3 Estátua Equestre do General Osório 2.4 Estátua do Almirante Negro 2.5 Piskate 3 Transportes 3.1 Modais de transporte público 3.2 Elevado da Perimetral 3.3 Mergulhão da Praça XV 4 Pontos de interesse 5 Feiras 5.1 Feira de An

In [29]:
gpt4_03_score_2_9_df.to_csv(os.path.join(ANNOTATIONS_FOLDER, "gpt_4_evaluations_240_samples_prompt_v2.9.tsv"), sep='\t', index=False)

In [72]:
gpt4_cot_2_6_df = pd.concat([gpt4_1st_cot_2_6_df, gpt4_2nd_cot_2_6_df], axis=0).reset_index(drop=True)

gpt4_cot_2_6_df

,doccano_id,query,passage,passage_id,score,reasoning,usage,cost,duration,02_score
0,4101,Onde está localizada a Praça XV de Novembro?,"Praça XV (Rio de Janeiro) – Wikipédia, a enciclopédia livre [ocultar] 5 línguas Praça XV (Rio de Janeiro) Origem: Wikipédia, a enciclopédia livre. Coordenadas: 22° 54′ 10″ S, 43° 10′ 24″ W Nota: Este artigo é sobre a praça da cidade do Rio de Janeiro. Para praças de outros lugares, veja Praça XV de Novembro. Esta página cita fontes, mas estas não cobrem todo o conteúdo. Ajude a inserir referências. Conteúdo não verificável poderá ser removido .— Encontre fontes: Google (notícias, livros e acadêmico)(Maio de 2019) A praça e as edificações das imediações. Trecho do Elevado da Perimetral que passava sobre a praça, em 2011. A Praça XV, também conhecida como Praça XV de Novembro, Praça Quinze de Novembro ou Praça Quinze, é uma praça situada no bairro do Centro, na Zona Central da cidade do Rio de Janeiro. Além de estar localizada na Rua Primeiro de Março, a praça integra a Orla Conde, um passeio público que margeia a Baía de Guanabara . A praça, que existe desde o século XVI, foi reinaugurada no dia 29 de maio de 2016 após ser revitalizada.[1] A revitalização da praça foi feita durante a gestão do prefeito Eduardo Paes no âmbito do Porto Maravilha, uma operação urbana que visa revitalizar a Zona Portuária do Rio de Janeiro.",clueweb22-pt0001-14-16263_0,3,"A passagem fornece uma resposta clara e direta à pergunta, indicando que a Praça XV de Novembro está localizada no bairro do Centro, na Zona Central da cidade do Rio de Janeiro. Além disso, a passagem fornece informações adicionais relevantes, como a rua em que a praça está localizada (Rua Primeiro de Março) e que a praça integra a Orla Conde, um passeio público que margeia a Baía de Guanabara.","{'prompt_tokens': 824, 'completion_tokens': 127, 'total_tokens': 951}",0.032340,9.277456,1
1,4102,Onde está localizada a Praça XV de Novembro?,"Rua XV de Novembro, 818 Centro, Pomerode, Santa Catarina 89107-000 Brasil Tela cheia O melhor por perto Restaurantes 100 dentro de 5 km Tarthurel Pizzaria 317 56 m $$ - $$$ • Italiana • Pizza Restaurante Torten Paradies 1.409 0.5 km $$ - $$$ • Docerias • Alemã • Brasileira Mon Cheri Bistro @ Casa Romena 106 0.7 km $$ - $$$ • Mediterrânea • Saudável • Leste europeia Veja tudo Atrações 24 dentro de 10 km Praça Jorge Lacerda 58 0.1 km Pontos de interesse Parque Municipal de Eventos 25 0.3 km Centros de convenção Parque Educativo Vila Encantada 1.057 0.9 km Parques Veja tudo Contribua Faça uma avaliação Carregue uma foto Avaliações Mais recente 4.5 722 avaliações Excelente 379 Muito boa 266 Razoável 68 Ruim 9 Horrível 1 Thais Leme, SP 161 contribuições Parada obrigatória para fotos mar de 2022 Quando eu via as fotos achava que era bem maior. O portal é muito bonito e tem um centro e informações turísticas. Estava no período da Osterfest, então estava todo com a decoração de Páscoa. Feita em 29 de abril de 2022 Esta avaliação representa a opinião subjetiva de um membro do Tripadvisor, e não da TripAdvisor LLC. daninoguei São Paulo, SP 2.143 contribuições MUITAS PESSOAS PARA TIRAR FOTOS mar de 2022 Portal turístico da cidade, local de parada para fotos com a representação da arquitetura local.",clueweb22-pt0000-27-16948_2,0,"A passagem não fornece informações sobre a localização da Praça XV de Novembro. A passagem menciona a Rua XV de Novembro, mas não a praça.","{'prompt_tokens': 840, 'completion_tokens': 52, 'total_tokens': 892}",0.028320,4.544968,0
2,4103,Onde está localizada a Praça XV de Novembro?,"A praça recebeu esse nome por homenagear o dia 15 de novembro de 1889, marcado pela Proclamação da República do Brasil. Nesse dia, foi instaurada a república federativa presidencialista no Brasil, em substituição à monarquia constitucional parlamentarista em vigor desde 1822. Índice 1 História 1.1 Período colonial 1.2 Século XIX 1.3 Século XX 1.4 Século XXI 2 Esculturas e Monumentos 2.1 Chafariz do Mestre Valentim 2.2 Est

In [73]:
gpt4_cot_2_6_df.to_csv(os.path.join(ANNOTATIONS_FOLDER, "gpt_4_evaluations_240_samples_prompt_v2.6.tsv"), sep='\t', index=False)

In [74]:
np.all(gpt4_cot_2_6_df['doccano_id'] == a3_df['doccano_id'])

True

# Check general agreement per annotator / LLM generation

In [ ]:
print(cohen_kappa_score(a1_df['score'], a2_df['score']))
print(cohen_kappa_score(a1_df['score'], a3_df['score']))
print(cohen_kappa_score(a2_df['score'], a3_df['score']))

In [ ]:
print(cohen_kappa_score(a1_df['score'], gpt4_df['score']))
print(cohen_kappa_score(a2_df['score'], gpt4_df['score']))
print(cohen_kappa_score(a3_df['score'], gpt4_df['score']))

In [ ]:
print(cohen_kappa_score(a1_df['03_score'], gpt4_df['03_score']))
print(cohen_kappa_score(a2_df['03_score'], gpt4_df['03_score']))
print(cohen_kappa_score(a3_df['03_score'], gpt4_df['03_score']))

### Put the results in a single table

In [ ]:
general_agreement_df = pd.DataFrame()

In [ ]:
general_agreement_df['comparisson'] = ['a1', 'a2', 'a3']

In [ ]:
general_agreement_df['a1'] = ["", 
                              cohen_kappa_score(a1_df['score'], a2_df['score']), 
                              cohen_kappa_score(a1_df['score'], a3_df['score'])]

general_agreement_df['a2'] = [cohen_kappa_score(a1_df['score'], a2_df['score']), 
                              "", 
                              cohen_kappa_score(a2_df['score'], a3_df['score'])]

general_agreement_df['a3'] = [cohen_kappa_score(a1_df['score'], a3_df['score']), 
                              cohen_kappa_score(a2_df['score'], a3_df['score']), 
                              ""]

general_agreement_df['GPT4'] = [cohen_kappa_score(a1_df['score'], gpt4_df['score']), 
                                cohen_kappa_score(a2_df['score'], gpt4_df['score']), 
                                cohen_kappa_score(a3_df['score'], gpt4_df['score'])]

In [ ]:
general_agreement_df

In [62]:
general_agreement_03_score_df = pd.DataFrame()

In [63]:
general_agreement_03_score_df['comparisson'] = ['a1 03 score', 'a2 03 score', 'a3 03 score']

In [65]:
general_agreement_03_score_df['a1 03 score'] = ["", cohen_kappa_score(a1_df['03_score'], a2_df['03_score']), cohen_kappa_score(a1_df['03_score'], a3_df['03_score'])]
general_agreement_03_score_df['a2 03 score'] = [cohen_kappa_score(a1_df['03_score'], a2_df['03_score']), "", cohen_kappa_score(a2_df['03_score'], a3_df['03_score'])]
general_agreement_03_score_df['a3 03 score'] = [cohen_kappa_score(a1_df['03_score'], a3_df['03_score']), cohen_kappa_score(a2_df['03_score'], a3_df['03_score']), ""]

general_agreement_03_score_df['GPT4 03 score'] = [cohen_kappa_score(a1_df['03_score'], gpt4_df['03_score']), 
                                                  cohen_kappa_score(a2_df['03_score'], gpt4_df['03_score']), 
                                                  cohen_kappa_score(a3_df['03_score'], gpt4_df['03_score'])]

general_agreement_03_score_df['GPT4 2nd 03 score reduced'] = [cohen_kappa_score(a1_df.iloc[120:240]['03_score'], gpt4_df.iloc[120:240]['03_score']), 
                                                              cohen_kappa_score(a2_df.iloc[120:240]['03_score'], gpt4_df.iloc[120:240]['03_score']), 
                                                              cohen_kappa_score(a3_df.iloc[120:240]['03_score'], gpt4_df.iloc[120:240]['03_score'])]

general_agreement_03_score_df['GPT4 2nd 03 score predicted 2.7'] = [cohen_kappa_score(a1_df.iloc[120:240]['03_score'], gpt4_2nd_03_score_2_7_df['score']), 
                                                                    cohen_kappa_score(a2_df.iloc[120:240]['03_score'], gpt4_2nd_03_score_2_7_df['score']), 
                                                                    cohen_kappa_score(a3_df.iloc[120:240]['03_score'], gpt4_2nd_03_score_2_7_df['score'])]

general_agreement_03_score_df['GPT4 2nd 03 score predicted 2.8'] = [cohen_kappa_score(a1_df.iloc[120:240]['03_score'], gpt4_2nd_03_score_2_8_df['score']), 
                                                                    cohen_kappa_score(a2_df.iloc[120:240]['03_score'], gpt4_2nd_03_score_2_8_df['score']), 
                                                                    cohen_kappa_score(a3_df.iloc[120:240]['03_score'], gpt4_2nd_03_score_2_8_df['score'])]

general_agreement_03_score_df['GPT4 2nd 03 score predicted 2.9'] = [cohen_kappa_score(a1_df.iloc[120:240]['03_score'], gpt4_2nd_03_score_2_9_df['score']), 
                                                                    cohen_kappa_score(a2_df.iloc[120:240]['03_score'], gpt4_2nd_03_score_2_9_df['score']), 
                                                                    cohen_kappa_score(a3_df.iloc[120:240]['03_score'], gpt4_2nd_03_score_2_9_df['score'])]

In [66]:
general_agreement_03_score_df

,comparisson,a1 03 score,a2 03 score,a3 03 score,GPT4 03 score,GPT4 2nd 03 score reduced,GPT4 2nd 03 score predicted 2.7,GPT4 2nd 03 score predicted 2.8,GPT4 2nd 03 score predicted 2.9
0,a1 03 score,,0.462148,0.435904,0.364067,0.309821,0.230688,0.135827,0.259653
1,a2 03 score,0.462148,,0.425263,0.353236,0.240262,0.175341,0.124684,0.210932
2,a3 03 score,0.435904,0.425263,,0.433279,0.362584,0.305838,0.245915,0.305392


In [67]:
cohen_kappa_score(gpt4_df.iloc[120:240]['03_score'], gpt4_2nd_03_score_2_9_df['score'])

0.7964592388423619

In [68]:
cohen_kappa_score(gpt4_df.iloc[120:240]['03_score'], gpt4_2nd_03_score_2_7_df['score'])

0.7204870301746955

In [69]:
cohen_kappa_score(gpt4_df.iloc[120:240]['03_score'], gpt4_2nd_03_score_2_8_df['score'])

0.550046870117696

### Check the classifications between the translated 3 score and the predicted 3 score

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
cohen_kappa_score(gpt4_df.iloc[120:240]['03_score'], gpt4_2nd_03_score_df['score'])

In [ ]:
cm_translated_versus_predicted = confusion_matrix(gpt4_df.iloc[120:240]['03_score'], gpt4_2nd_03_score_df['score'])

In [ ]:
cm_translated_versus_predicted

In [ ]:
np.sum(cm_translated_versus_predicted, axis=1)

In [ ]:
np.sum(cm_translated_versus_predicted, axis=0)

In [ ]:
np.unique(gpt4_df.iloc[120:240]['03_score'].to_numpy(), return_counts=True)

In [ ]:
cohen_kappa_score(gpt4_df.iloc[120:240]['03_score_2'], gpt4_2nd_03_score_df['score'])

In [ ]:
cm_translated2_versus_predicted = confusion_matrix(gpt4_df.iloc[120:240]['03_score_2'], gpt4_2nd_03_score_df['score'])

In [ ]:
cm_translated2_versus_predicted

In [ ]:
np.sum(cm_translated2_versus_predicted, axis=1)

### Which zeros the model predicted as 1 and 2?

In [ ]:
gpt4_2nd_03_score_df[(gpt4_df.iloc[120:240]['03_score'] == 0).reset_index(drop=True)]

## Two score correlations

In [31]:
general_agreement_02_score_df = pd.DataFrame()

general_agreement_02_score_df['comparisson'] = ['a1 02 score', 'a2 02 score', 'a3 02 score']

In [32]:
general_agreement_02_score_df['a1 02 score'] = [np.nan, cohen_kappa_score(a1_df['02_score'], a2_df['02_score']), cohen_kappa_score(a1_df['02_score'], a3_df['02_score'])]
general_agreement_02_score_df['a2 02 score'] = [cohen_kappa_score(a1_df['02_score'], a2_df['02_score']), np.nan, cohen_kappa_score(a2_df['02_score'], a3_df['02_score'])]
general_agreement_02_score_df['a3 02 score'] = [cohen_kappa_score(a1_df['02_score'], a3_df['02_score']), cohen_kappa_score(a2_df['02_score'], a3_df['02_score']), np.nan]

general_agreement_02_score_df['GPT4 02 score'] = [cohen_kappa_score(a1_df['02_score'], gpt4_df['02_score']), 
                                                  cohen_kappa_score(a2_df['02_score'], gpt4_df['02_score']), 
                                                  cohen_kappa_score(a3_df['02_score'], gpt4_df['02_score'])]

general_agreement_02_score_df['GPT4 2nd 02 score reduced'] = [cohen_kappa_score(a1_df.iloc[120:240]['02_score'], gpt4_df.iloc[120:240]['02_score']), 
                                                              cohen_kappa_score(a2_df.iloc[120:240]['02_score'], gpt4_df.iloc[120:240]['02_score']), 
                                                              cohen_kappa_score(a3_df.iloc[120:240]['02_score'], gpt4_df.iloc[120:240]['02_score'])]

general_agreement_02_score_df['GPT4 2nd 02 score predicted 2.7'] = [cohen_kappa_score(a1_df.iloc[120:240]['02_score'], gpt4_2nd_03_score_2_7_df['02_score']), 
                                                                    cohen_kappa_score(a2_df.iloc[120:240]['02_score'], gpt4_2nd_03_score_2_7_df['02_score']), 
                                                                    cohen_kappa_score(a3_df.iloc[120:240]['02_score'], gpt4_2nd_03_score_2_7_df['02_score'])]

general_agreement_02_score_df['GPT4 2nd 02 score predicted 2.8'] = [cohen_kappa_score(a1_df.iloc[120:240]['02_score'], gpt4_2nd_03_score_2_8_df['02_score']), 
                                                                    cohen_kappa_score(a2_df.iloc[120:240]['02_score'], gpt4_2nd_03_score_2_8_df['02_score']), 
                                                                    cohen_kappa_score(a3_df.iloc[120:240]['02_score'], gpt4_2nd_03_score_2_8_df['02_score'])]

general_agreement_02_score_df['GPT4 2nd 02 score predicted 2.9'] = [cohen_kappa_score(a1_df.iloc[120:240]['02_score'], gpt4_2nd_03_score_2_9_df['02_score']), 
                                                                    cohen_kappa_score(a2_df.iloc[120:240]['02_score'], gpt4_2nd_03_score_2_9_df['02_score']), 
                                                                    cohen_kappa_score(a3_df.iloc[120:240]['02_score'], gpt4_2nd_03_score_2_9_df['02_score'])]

general_agreement_02_score_df['GPT4 02 score predicted 2.7'] = [cohen_kappa_score(a1_df['02_score'], gpt4_03_score_2_7_df['02_score']), 
                                                                cohen_kappa_score(a2_df['02_score'], gpt4_03_score_2_7_df['02_score']), 
                                                                cohen_kappa_score(a3_df['02_score'], gpt4_03_score_2_7_df['02_score'])]

general_agreement_02_score_df['GPT4 02 score predicted 2.9'] = [cohen_kappa_score(a1_df['02_score'], gpt4_03_score_2_9_df['02_score']), 
                                                                cohen_kappa_score(a2_df['02_score'], gpt4_03_score_2_9_df['02_score']), 
                                                                cohen_kappa_score(a3_df['02_score'], gpt4_03_score_2_9_df['02_score'])]

In [37]:
general_agreement_02_score_df['GPT4 2nd cot 2.6'] = [cohen_kappa_score(a1_df.iloc[120:240]['02_score'], gpt4_2nd_cot_2_6_df['02_score']), 
                                                     cohen_kappa_score(a2_df.iloc[120:240]['02_score'], gpt4_2nd_cot_2_6_df['02_score']), 
                                                     cohen_kappa_score(a3_df.iloc[120:240]['02_score'], gpt4_2nd_cot_2_6_df['02_score'])]


In [75]:
general_agreement_02_score_df['GPT4 cot 2.6'] = [cohen_kappa_score(a1_df['02_score'], gpt4_cot_2_6_df['02_score']), 
                                                 cohen_kappa_score(a2_df['02_score'], gpt4_cot_2_6_df['02_score']), 
                                                 cohen_kappa_score(a3_df['02_score'], gpt4_cot_2_6_df['02_score'])]


In [76]:
general_agreement_02_score_df

,comparisson,a1 02 score,a2 02 score,a3 02 score,GPT4 02 score,GPT4 2nd 02 score reduced,GPT4 2nd 02 score predicted 2.7,GPT4 2nd 02 score predicted 2.8,GPT4 2nd 02 score predicted 2.9,GPT4 02 score predicted 2.7,GPT4 02 score predicted 2.9,GPT4 2nd cot 2.6,GPT4 cot 2.6
0,a1 02 score,NaN,0.533333,0.558333,0.441667,0.466307,0.509859,0.481894,0.538462,0.450000,0.475000,0.535912,0.500000
1,a2 02 score,0.533333,NaN,0.551797,0.484144,0.361702,0.445378,0.382304,0.441624,0.381107,0.390987,0.435216,0.389271
2,a3 02 score,0.558333,0.551797,NaN,0.525926,0.504556,0.521978,0.550998,0.493464,0.471545,0.465046,0.497768,0.496907


In [77]:
pd.concat([general_agreement_02_score_df, pd.DataFrame(data=[['mean'] + general_agreement_02_score_df.iloc[:, 1:].mean().to_list()], columns=general_agreement_02_score_df.columns)])

,comparisson,a1 02 score,a2 02 score,a3 02 score,GPT4 02 score,GPT4 2nd 02 score reduced,GPT4 2nd 02 score predicted 2.7,GPT4 2nd 02 score predicted 2.8,GPT4 2nd 02 score predicted 2.9,GPT4 02 score predicted 2.7,GPT4 02 score predicted 2.9,GPT4 2nd cot 2.6,GPT4 cot 2.6
0,a1 02 score,NaN,0.533333,0.558333,0.441667,0.466307,0.509859,0.481894,0.538462,0.450000,0.475000,0.535912,0.500000
1,a2 02 score,0.533333,NaN,0.551797,0.484144,0.361702,0.445378,0.382304,0.441624,0.381107,0.390987,0.435216,0.389271
2,a3 02 score,0.558333,0.551797,NaN,0.525926,0.504556,0.521978,0.550998,0.493464,0.471545,0.465046,0.497768,0.496907
0,mean,0.545833,0.542565,0.555065,0.483912,0.444188,0.492405,0.471732,0.491183,0.434217,0.443677,0.489632,0.462059


### Check the classifications between the translated 2 score from original 4-score predictions and the last 3-score predictions for the first set of questions

In [ ]:
from sklearn.metrics import confusion_matrix

In [48]:
cohen_kappa_score(gpt4_df.iloc[0:120]['02_score'], gpt4_03_score_2_9_df.iloc[0:120]['02_score'])

0.607843137254902

In [49]:
cohen_kappa_score(gpt4_df.iloc[0:120]['03_score'], gpt4_03_score_2_9_df.iloc[0:120]['score'])

0.8307291666666667

In [50]:
cohen_kappa_score(gpt4_df.iloc[120:240]['02_score'], gpt4_03_score_2_9_df.iloc[120:240]['02_score'])

0.643146796431468

In [52]:
cohen_kappa_score(gpt4_df.iloc[120:240]['03_score'], gpt4_03_score_2_9_df.iloc[120:240]['score'])

0.7964592388423619

In [57]:
print(cohen_kappa_score(a1_df.iloc[0:120]['02_score'], gpt4_03_score_2_9_df.iloc[0:120]['02_score']))
print(cohen_kappa_score(a1_df.iloc[120:240]['02_score'], gpt4_03_score_2_9_df.iloc[120:240]['02_score']))

0.4285714285714286
0.5384615384615384


In [55]:
print(cohen_kappa_score(a2_df.iloc[0:120]['02_score'], gpt4_03_score_2_9_df.iloc[0:120]['02_score']))
print(cohen_kappa_score(a2_df.iloc[120:240]['02_score'], gpt4_03_score_2_9_df.iloc[120:240]['02_score']))

0.36363636363636365
0.4416243654822335


In [56]:
print(cohen_kappa_score(a3_df.iloc[0:120]['02_score'], gpt4_03_score_2_9_df.iloc[0:120]['02_score']))
print(cohen_kappa_score(a3_df.iloc[120:240]['02_score'], gpt4_03_score_2_9_df.iloc[120:240]['02_score']))

0.43639291465378416
0.4934640522875817


In [59]:
print(cohen_kappa_score(a1_df.iloc[0:120]['03_score'], gpt4_03_score_2_9_df.iloc[0:120]['score']))
print(cohen_kappa_score(a1_df.iloc[120:240]['03_score'], gpt4_03_score_2_9_df.iloc[120:240]['score']))

0.4022891055532004
0.25965322837995963


In [60]:
print(cohen_kappa_score(a2_df.iloc[0:120]['03_score'], gpt4_03_score_2_9_df.iloc[0:120]['score']))
print(cohen_kappa_score(a2_df.iloc[120:240]['03_score'], gpt4_03_score_2_9_df.iloc[120:240]['score']))

0.44762684124386254
0.21093188122880924


In [61]:
print(cohen_kappa_score(a3_df.iloc[0:120]['03_score'], gpt4_03_score_2_9_df.iloc[0:120]['score']))
print(cohen_kappa_score(a3_df.iloc[120:240]['03_score'], gpt4_03_score_2_9_df.iloc[120:240]['score']))

0.44288490606780395
0.3053917890449136


In [ ]:
cm_translated_versus_predicted = confusion_matrix(gpt4_df.iloc[0:120]['03_score'], gpt4_2nd_cot_2_6_df['score'])

cm_translated_versus_predicted

In [ ]:
np.sum(cm_translated_versus_predicted, axis=1)

np.sum(cm_translated_versus_predicted, axis=0)

np.unique(gpt4_df.iloc[120:240]['03_score'].to_numpy(), return_counts=True)

cohen_kappa_score(gpt4_df.iloc[120:240]['03_score_2'], gpt4_2nd_03_score_df['score'])

cm_translated2_versus_predicted = confusion_matrix(gpt4_df.iloc[120:240]['03_score_2'], gpt4_2nd_03_score_df['score'])

cm_translated2_versus_predicted

np.sum(cm_translated2_versus_predicted, axis=1)

In [ ]:
from scipy.stats import pearsonr

In [ ]:
general_agreement_pearson_df = pd.DataFrame()

general_agreement_pearson_df['comparisson'] = ['a1', 'a2', 'a3']

general_agreement_pearson_df['a1'] = ["", 
                              pearsonr(a1_df['score'], a2_df['score'])[0], 
                              pearsonr(a1_df['score'], a3_df['score'])[0]]

general_agreement_pearson_df['a2'] = [pearsonr(a1_df['score'], a2_df['score'])[0], 
                              "", 
                              pearsonr(a2_df['score'], a3_df['score'])[0]]

general_agreement_pearson_df['a3'] = [pearsonr(a1_df['score'], a3_df['score'])[0], 
                              pearsonr(a2_df['score'], a3_df['score'])[0], 
                              ""]

general_agreement_pearson_df['GPT4'] = [pearsonr(a1_df['score'], gpt4_df['score'])[0], 
                                pearsonr(a2_df['score'], gpt4_df['score'])[0], 
                                pearsonr(a3_df['score'], gpt4_df['score'])[0]]

In [ ]:
general_agreement_pearson_df

In [ ]:
general_agreement_03_score_pearson_df = pd.DataFrame()

general_agreement_03_score_pearson_df['comparisson'] = ['a1 03 score', 'a2 03 score', 'a3 03 score']

general_agreement_03_score_pearson_df['a1 03 score'] = ["", 
                                                        pearsonr(a1_df['03_score'], a2_df['03_score'])[0], 
                                                        pearsonr(a1_df['03_score'], a3_df['03_score'])[0]]

general_agreement_03_score_pearson_df['a2 03 score'] = [pearsonr(a1_df['03_score'], a2_df['03_score'])[0], 
                                                        "", 
                                                        pearsonr(a2_df['03_score'], a3_df['03_score'])[0]]

general_agreement_03_score_pearson_df['a3 03 score'] = [pearsonr(a1_df['03_score'], a3_df['03_score'])[0], 
                                                        pearsonr(a2_df['03_score'], a3_df['03_score'])[0], ""]

general_agreement_03_score_pearson_df['GPT4 03 score'] = [pearsonr(a1_df['03_score'], gpt4_df['03_score'])[0], 
                                                  pearsonr(a2_df['03_score'], gpt4_df['03_score'])[0], 
                                                  pearsonr(a3_df['03_score'], gpt4_df['03_score'])[0]]

general_agreement_03_score_pearson_df['GPT4 2nd 03 score reduced'] = [pearsonr(a1_df.iloc[120:240]['03_score'], gpt4_df.iloc[120:240]['03_score'])[0], 
                                                              pearsonr(a2_df.iloc[120:240]['03_score'], gpt4_df.iloc[120:240]['03_score'])[0], 
                                                              pearsonr(a3_df.iloc[120:240]['03_score'], gpt4_df.iloc[120:240]['03_score'])[0]]

general_agreement_03_score_pearson_df['GPT4 2nd 03 score predicted'] = [pearsonr(a1_df.iloc[120:240]['03_score'], gpt4_2nd_03_score_df['score'])[0], 
                                                                pearsonr(a2_df.iloc[120:240]['03_score'], gpt4_2nd_03_score_df['score'])[0], 
                                                                pearsonr(a3_df.iloc[120:240]['03_score'], gpt4_2nd_03_score_df['score'])[0]]

In [ ]:
general_agreement_03_score_pearson_df

In [ ]:
gpt4_df.iloc[120:240]['score'].to_numpy()

In [ ]:
gpt4_df.iloc[120:240]['03_score'].to_numpy()

In [ ]:
gpt4_2nd_03_score_df['score'].to_numpy()

# Check agreement per question

In [ ]:
res_a1_a2 = check_agreement_per_questions(a1_df, a2_df, "_a1_a2")
res_a1_a3 = check_agreement_per_questions(a1_df, a3_df, "_a1_a3")
res_a2_a3 = check_agreement_per_questions(a2_df, a3_df, "_a2_a3")

In [ ]:
res_a1_gpt4 = check_agreement_per_questions(a1_df, gpt4_df, "_a1_gpt4")
res_a1_gpt4_n5 = check_agreement_per_questions(a1_df, gpt4_n5_df, "_a1_gpt4_n5")
res_a1_gpt4_n10 = check_agreement_per_questions(a1_df, gpt4_n10_df, "_a1_gpt4_n10")

In [ ]:
res_a2_gpt4 = check_agreement_per_questions(a2_df, gpt4_df, "_a2_gpt4")
res_a2_gpt4_n5 = check_agreement_per_questions(a2_df, gpt4_n5_df, "_a2_gpt4_n5")
res_a2_gpt4_n10 = check_agreement_per_questions(a2_df, gpt4_n10_df, "_a2_gpt4_n10")

In [ ]:
res_a3_gpt4 = check_agreement_per_questions(a3_df, gpt4_df, "_a3_gpt4")
res_a3_gpt4_n5 = check_agreement_per_questions(a3_df, gpt4_n5_df, "_a3_gpt4_n5")
res_a3_gpt4_n10 = check_agreement_per_questions(a3_df, gpt4_n10_df, "_a3_gpt4_n10")

In [ ]:
human_combined_res = pd.concat([res_a1_a2[1], res_a1_a3[1].iloc[:, -1], res_a2_a3[1].iloc[:, -1]], axis=1)

In [ ]:
human_combined_res['cohen_kappa_mean'] = human_combined_res.iloc[:, 1:].mean(axis=1)
human_combined_res['cohen_kappa_std'] = human_combined_res.iloc[:, 1:].std(axis=1)

In [ ]:
human_gpt4_combined_res = pd.concat([res_a1_gpt4[1], res_a2_gpt4[1].iloc[:, -1], res_a3_gpt4[1].iloc[:, -1]], axis=1)

In [ ]:
human_gpt4_combined_res['cohen_kappa_mean'] = human_gpt4_combined_res.iloc[:, 1:].mean(axis=1)
human_gpt4_combined_res['cohen_kappa_std'] = human_gpt4_combined_res.iloc[:, 1:].std(axis=1)

In [ ]:
human_gpt4_n5_combined_res = pd.concat([res_a1_gpt4_n5[1], res_a2_gpt4_n5[1].iloc[:, -1], res_a3_gpt4_n5[1].iloc[:, -1]], axis=1)

In [ ]:
human_gpt4_n5_combined_res['cohen_kappa_mean'] = human_gpt4_n5_combined_res.iloc[:, 1:].mean(axis=1)
human_gpt4_n5_combined_res['cohen_kappa_std'] = human_gpt4_n5_combined_res.iloc[:, 1:].std(axis=1)

In [ ]:
human_gpt4_n10_combined_res = pd.concat([res_a1_gpt4_n10[1], res_a2_gpt4_n10[1].iloc[:, -1], res_a3_gpt4_n10[1].iloc[:, -1]], axis=1)

In [ ]:
human_gpt4_n10_combined_res['cohen_kappa_mean'] = human_gpt4_n10_combined_res.iloc[:, 1:].mean(axis=1)
human_gpt4_n10_combined_res['cohen_kappa_std'] = human_gpt4_n10_combined_res.iloc[:, 1:].std(axis=1)

In [ ]:
all_questions_comparisson_df = pd.concat([human_combined_res.iloc[:, [0, 5, 4]], 
                                          human_gpt4_combined_res.iloc[:, [4, 5]],
                                          human_gpt4_n5_combined_res.iloc[:, [4, 5]],
                                          human_gpt4_n10_combined_res.iloc[:, [4, 5]]], axis=1)

In [ ]:
all_questions_comparisson_df.columns

In [ ]:
all_questions_comparisson_df.columns=['query', 'human_human_mean', 'human_human_std', 'human_gpt4_mean', 'human_gpt4_std', 'human_gpt4_n5_mean', 'human_gpt4_n5_std', 'human_gpt4_n10_mean', 'human_gpt4_n10_std']

### Agreement per question comparisson

In [ ]:
all_questions_comparisson_df

In [ ]:
all_questions_comparisson_df.iloc[:, [1, 3, 5, 7]].mean()

In [ ]:
all_questions_comparisson_df['max'] = all_questions_comparisson_df.iloc[:, [1, 3, 5, 7]].idxmax(axis='columns')

In [ ]:
np.unique(all_questions_comparisson_df['max'].to_numpy(), return_counts=True)

### Agreement per question per annotator / LLM generation

In [ ]:
indexes = [
    np.repeat(human_combined_res['query'].to_numpy(), 3),
    np.tile(['a1', 'a2', 'a3'], 24)
]

In [ ]:
comparisson_results_a1 = []
comparisson_results_a2 = []
comparisson_results_a3 = []

for i, row in human_combined_res.iterrows():
    comparisson_results_a1.append(None)
    comparisson_results_a1.append(row['cohen_kappa_a1_a2'])
    comparisson_results_a1.append(row['cohen_kappa_a1_a3'])
    
    comparisson_results_a2.append(row['cohen_kappa_a1_a2'])
    comparisson_results_a2.append(None)
    comparisson_results_a2.append(row['cohen_kappa_a2_a3'])    
    
    comparisson_results_a3.append(row['cohen_kappa_a1_a3'])
    comparisson_results_a3.append(row['cohen_kappa_a2_a3'])        
    comparisson_results_a3.append(None)

In [ ]:
all_comparissons_per_question_df = pd.DataFrame(comparisson_results_a1, index=indexes, columns=['a1'])

In [ ]:
all_comparissons_per_question_df['a2'] = comparisson_results_a2
all_comparissons_per_question_df['a3'] = comparisson_results_a3

In [ ]:
comparisson_results_gpt4 = []
comparisson_results_gpt4_n5 = []
comparisson_results_gpt4_n10 = []

for i in range(human_gpt4_combined_res.shape[0]):
    comparisson_results_gpt4.append(human_gpt4_combined_res.iloc[i, 1])
    comparisson_results_gpt4.append(human_gpt4_combined_res.iloc[i, 2])
    comparisson_results_gpt4.append(human_gpt4_combined_res.iloc[i, 3])
    
    comparisson_results_gpt4_n5.append(human_gpt4_n5_combined_res.iloc[i, 1])
    comparisson_results_gpt4_n5.append(human_gpt4_n5_combined_res.iloc[i, 2])
    comparisson_results_gpt4_n5.append(human_gpt4_n5_combined_res.iloc[i, 3])

    comparisson_results_gpt4_n10.append(human_gpt4_n10_combined_res.iloc[i, 1])
    comparisson_results_gpt4_n10.append(human_gpt4_n10_combined_res.iloc[i, 2])
    comparisson_results_gpt4_n10.append(human_gpt4_n10_combined_res.iloc[i, 3])
                                    

In [ ]:
all_comparissons_per_question_df['gpt4'] = comparisson_results_gpt4
all_comparissons_per_question_df['gpt4_n5'] = comparisson_results_gpt4_n5
all_comparissons_per_question_df['gpt4_n10'] = comparisson_results_gpt4_n10

In [ ]:
all_comparissons_per_question_df['mean_human'] = all_comparissons_per_question_df.iloc[:,:3].mean(axis=1)
all_comparissons_per_question_df['std_human'] = all_comparissons_per_question_df.iloc[:, :3].std(axis=1)

In [ ]:
all_comparissons_per_question_df['mean_gpt4'] = all_comparissons_per_question_df.iloc[:, 3:6].mean(axis=1)
all_comparissons_per_question_df['std_gpt4'] = all_comparissons_per_question_df.iloc[:, 3:6].std(axis=1)

In [ ]:
all_comparissons_per_question_df

In [ ]:
all_comparissons_per_question_df['mean_human'].mean()

In [ ]:
all_comparissons_per_question_df['mean_gpt4'].mean()

## Build boxplot comparing the correlations per question

In [ ]:
all_combined = []

In [ ]:
for i, row in human_combined_res.iterrows():
    all_combined.append({"query": row['query'],
                         "first": row['cohen_kappa_a1_a2'],
                         "second": row['cohen_kappa_a1_a3'],
                         "third": row['cohen_kappa_a2_a3'],
                         "type": "human_human"})
    
    all_combined.append({"query": human_gpt4_combined_res.iloc[i, 0],
                         "first": human_gpt4_combined_res.iloc[i, 1],
                         "second": human_gpt4_combined_res.iloc[i, 2],
                         "third": human_gpt4_combined_res.iloc[i, 3],
                         "type": "human_gpt4"})
    
    all_combined.append({"query": human_gpt4_n5_combined_res.iloc[i, 0],
                         "first": human_gpt4_n5_combined_res.iloc[i, 1],
                         "second": human_gpt4_n5_combined_res.iloc[i, 2],
                         "third": human_gpt4_n5_combined_res.iloc[i, 3],
                         "type": "human_gpt4_n5"})
    
    all_combined.append({"query": human_gpt4_n10_combined_res.iloc[i, 0],
                         "first": human_gpt4_n10_combined_res.iloc[i, 1],
                         "second": human_gpt4_n10_combined_res.iloc[i, 2],
                         "third": human_gpt4_n10_combined_res.iloc[i, 3],
                         "type": "human_gpt4_n10"})    

In [ ]:
all_combined_df = pd.DataFrame(all_combined)

In [ ]:
plot_correlation_data(all_combined_df.iloc[::-1], all_combined_df.columns[1:4], "Cohen Kappa por questão")